In [1]:
from __future__ import division
import os
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')
import statsmodels.api as sm
% matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn import linear_model

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: license expired

    the package numpy 1.10.2 was published on 2015-12-14,
    however the mkl license expiration date is 2015-08-05.
    You may be able to run earlier versions of numpy using your
    current license.  A new license can be purchased at: http://continuum.io
    To revert to an earlier set of conda packages, use:
    $ conda list --revisions
    ...
    $ conda install --revision <REVISION NUMBER>

    
Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: license expired

    the package numpy 1.10.2 was published on 2015-12-14,
    however the mkl license expiration date is 2015-08-05.
    You may be able to run earlier versions of numpy using your
    current license.  A new license can be purchased at: http://continuum.io
    To revert to an earlier set of conda packages, use:
    $ conda list --revisions
    ...
    $ conda install --revision <REVISION NUMBER>

    


In [2]:
base_dir = '/om/user/zqi/projects/slbeh/cogscigame/data_structured/concatenated'

In [3]:
def lm(X, y):
    if len(X.shape) == 1:
        X = X.reshape(-1, 1)
    clf = Pipeline([
        ('scale', StandardScaler()),
        ('lm', linear_model.LinearRegression())
    ])
    clf.fit(X, y)
    coef = clf.named_steps["lm"].coef_
    return coef

def poly(X, y):
    if len(X.shape) == 1:
        X = X.reshape(-1, 1)
    clf = Pipeline([
        ('poly', PolynomialFeatures(degree=3)),
        ('scale', StandardScaler()),
        ('lm', linear_model.LinearRegression())
    ])
    clf.fit(X, y)
    return clf.named_steps["lm"].coef_
    
def group_familiarization_summary(path):
    data = pd.read_csv(path)
    pars = data.groupby('id')
    summary_mat = np.zeros((len(pars), 6), dtype=object)
    for idx, (name, group) in enumerate(pars):
        summary_mat[idx, 0] = name
        summary_mat[idx, 1] = lm(
            group.trial_index.values.astype(float), 
            group.rt.values.astype(float)
        )[0]
        summary_mat[idx, 2] = group.rt.mean()
        coefs = poly(
            group.trial_index.values.astype(float),
            group.rt.values.astype(float)
                    )
        summary_mat[idx, 3] = coefs[1]
        summary_mat[idx, 4] = coefs[2]
        summary_mat[idx, 5] = coefs[3]
    cols = ['id','lm_coef_rt', 'mean_rt', 'poly1', 'poly2', 'poly3']
    return pd.DataFrame(summary_mat, columns=cols)

def group_fam_target_summary(path):
    data = pd.read_csv(path)
    pars = data.groupby('id')
    summary_mat = np.zeros((len(pars), 2), dtype=object)
    for idx, (name, group) in enumerate(pars):
        hit = 0
        trials = len(group.stimulus)
        for i in range(trials):
            if group.targ.values[i] in group.stimulus.values[i]:
                hit = hit + 1
        summary_mat[idx, 0] = name
        summary_mat[idx, 1] = hit/trials
    return pd.DataFrame(summary_mat, columns=['id','fam_acc'])

def group_test_summary(path):
    data = pd.read_csv(path)
    pars = data.groupby('id')
    summary_mat = np.zeros((len(pars), 2), dtype=object)
    for idx, (name, group) in enumerate(pars):
        summary_mat[idx, 0] = name
        summary_mat[idx, 1] = (group.par_answer == group.expected_answer).mean()
    return pd.DataFrame(summary_mat, columns=['id','acc'])

def combine_summary(rt, acc, famacc):
    out = rt.set_index('id').loc[acc['id']]
    out['id_rt'] = out.index.values
    out.index = [idx for idx in range(out.shape[0])]
    out['id_acc'] = acc['id'].values
    out['id_famacc'] = famacc['id'].values
    out['acc'] = acc['acc'].values
    out['fam_acc'] = famacc['fam_acc'].values
    if not np.all(out.id_acc == out.id_rt):
        print("IDs DO NOT MATCH")
    if not np.all(out.id_famacc == out.id_rt):
        print("IDs DO NOT MATCH")
    return out

In [4]:
vslfamphase=os.path.join(base_dir, 'adults_vsl_tbt_famphase.csv')
vsltest=os.path.join(base_dir, 'adults_vsl_tbt_testphase.csv')
vslfamacc=os.path.join(base_dir, 'adults_vsl_tbt_famphase_falsePos.csv')
tslfamphase=os.path.join(base_dir, 'adults_tsl_tbt_famphase.csv')
tsltest=os.path.join(base_dir, 'adults_tsl_tbt_testphase.csv')
tslfamacc=os.path.join(base_dir, 'adults_tsl_tbt_famphase_falsePos.csv')

In [ ]:
data = pd.read_csv(tslfamphase)
pars = data.groupby('id')
summary_mat = np.zeros((len(pars), 2), dtype=object
                       for idx, (name, group) in enumerate(pars):
        hit =    trials = len(group.stimulus)
        for i in range(trials)    if group.targ.values[i] in group.stimulus.values[i]:
                hit = hit + summary_mat[idx, 0] = name
        summary_mat[idx, 1] = hit/trials
    return pd.DataFrame(summary_mat, columns=['id','fam_acc'])

In [14]:
famacc=group_fam_target_summary(tslfamacc)
rt = group_familiarization_summary(tslfamphase)
acc=group_test_summary(tsltest)

TypeError: argument of type 'float' is not iterable

In [13]:
combine_summary(rt,acc,famacc).to_csv(os.path.join(base_dir,'summary/summary_adults_vsl.csv'))

In [13]:
famacc=group_fam_target_summary(vslfamacc)
rt = group_familiarization_summary(g[0])
acc = group_test_summary(g[1])
combine_summary(rt, acc, famacc).to_csv(os.path.join(base_dir,'summary/summary_adults_{}.csv'.format(g[0][73:76])))

ValueError: Length of values does not match length of index

In [11]:
for g in [
    (vslfamphase, vsltest, vslfamacc),
    (tslfamphase, tsltest, tslfamacc)
]:
    rt = group_familiarization_summary(g[0])
    acc = group_test_summary(g[1])
    famacc = group_fam_target_summary(g[2])
    combine_summary(rt, acc, famacc).to_csv(os.path.join(base_dir,'summary/summary_adults_{}.csv'.format(g[0][73:76])))

print('')

TypeError: argument of type 'float' is not iterable

In [6]:
len(famacc)

33

In [29]:
print(famacc)

       id   fam_acc
0   SLA01         1
1   SLA02         1
2   SLA03         1
3   SLA04         1
4   SLA05  0.954545
5   SLA06         1
6   SLA10      0.96
7   SLA11  0.958333
8   SLA12         1
9   SLA14         1
10  SLA15         1
11  SLA18         1
12  SLA19         1
13  SLA20      0.96
14  SLA21     0.875
15  SLA22      0.96
16  SLA23      0.96
17  SLA24  0.958333
18  SLA25         1
19  SLA26  0.851852
20  SLA27         1
21  SLA28  0.958333
22  SLA29         1
23  SLA30  0.913043
24  SLA31         1
25  SLA32  0.958333
26  SLA33      0.96
27  SLA34         1
28  SLA35         1
29  SLA36         1
30  SLA37         1
31  sla16  0.958333
32  sla17         1


In [7]:
print(acc)

       id      acc
0   LSA14  0.90625
1   SLA01  0.78125
2   SLA02   0.4375
3   SLA03  0.96875
4   SLA04  0.65625
5   SLA06    0.875
6   SLA07  0.84375
7   SLA08  0.46875
8   SLA10   0.6875
9   SLA11  0.71875
10  SLA12  0.90625
11  SLA13   0.9375
12  SLA15   0.9375
13  SLA18        1
14  SLA19  0.40625
15  SLA20        1
16  SLA21   0.5625
17  SLA22    0.875
18  SLA23  0.96875
19  SLA24    0.875
20  SLA25  0.90625
21  SLA26  0.40625
22  SLA27        1
23  SLA28  0.96875
24  SLA29    0.875
25  SLA30  0.96875
26  SLA31     0.75
27  SLA32  0.53125
28  SLA33  0.53125
29  SLA34   0.6875
30  SLA35   0.9375
31  SLA36  0.59375
32  SLA37  0.84375
33  SLAO5  0.65625
34  sla16        1
35  sla17  0.71875
